# Converting PlanetScope Imagery from Radiance to Reflectance

Planet Labs' analytic data products (both Rapideye and Dove) are reported in units of radiance: $W*m^{-2}*sr^{-1}$. That means that every pixel in an analytic tiff has a natural language interpretation: "How much light was captured over this spot of ground?"

But over the course of a day or year, the number of photons that the sun shines on the scene rises and falls. If you naively compare the radiance values of two scenes over the same spot on Earth from the same satellite but a few weeks (or even just hours!) apart, you will likely find dramatic radiance differences **even if nothing on the ground has changed!**

In addition to this variation, each of Planet Labs' 150+ satllites have small amounts of variation in their spectral filters which yields slight differences in radiance measurements, even from two satellites taking pictures of the same exact place at the same exact moment!

![Illumination](illumination.png)

To correct all this radiance variation you would have to do a lot of math using the exact location and local time of day to find the angle to the sun and sun-earth distance, then compute a solar irradiance model to estimate how many photons of each band are present in the image, and finally convolve that spectrum with the spectral response of the individual satellite to yield the number of photons of each band that are actually recorded by that sensor. Dividing by this number normalizes the measured brightness to the brightness of the sun at that time and place through that particular filter, yielding a much more comparable number: reflectance.

Top of Atmosphere Reflectance is extremely useful because it is an apples-to-apples comparable number from any satellite over any location that does not change with time of day or time of year **unless the content of the scene changes.** It is very commonly used in GIS applications which compute spectral indices such as NDVI or NDWI. 

It is so broadly useful that Planet does this math for you and provides all the coefficients necessary to convert a radiance image into a reflectance image!

$$
\begin{equation*}
\mathbf{img}_{reflectance} =  \begin{vmatrix}
a \\
b \\
c \\
d 
\end{vmatrix}
\times \mathbf{img}_{radiance}
\end{equation*}
$$


The four coefficients $a, b, c, d$ are calculated and provided with every analytic image that Planet provides and can be used as simple scaling factors to convert from radiance to reflectance. Their values change with the image's local 
time of day and time of year, and do so uniquely per satellite.

**In this guide, you'll perform a basic Radiance to Reflectance calculation on PlanetScope imagery using just a few lines of Python. Here are the steps:**

1. Download a PlanetScope image
2. Extract data from each spectral band
3. Extract the coefficients
4. Convert Radiance to Reflectance
5. Save the Reflectance
6. Apply a color scheme to visualize reflectance on a map

### Requirements
- Python 2.7 or 3+
- [Planet's Python Client](https://www.planet.com/docs/api-quickstart-examples/cli/)
- [rasterio](https://github.com/mapbox/rasterio)
- [numpy](http://www.numpy.org/)
- [matplotlib](https://matplotlib.org/)
- [Planet API Key](https://www.planet.com/account/#/), stored as environment variable `$PL_API_KEY`.
- [Planet 4-Band Imagery](https://developers.planet.com/docs/data/psscene/) with the following specifications: `item-type`: `PSOrthoTile`, `REOrthoTile`, or `PSScene`; `asset-type`: `ortho_analytic_4b`, or `basic_analytic_4b`

## Step 1. Download a PlanetScope Image 

First, you're going to download a [4-band PlanetScope satellite image](https://developers.planet.com/docs/data/psscene/) of agricultural land in California's Central Valley, captured in mid July 2017 (`item-id`: `20170623_180038_0f34`). You can do this using [Planet's Python client](https://www.planet.com/docs/api-quickstart-examples/cli/) to interact with our Data API, or by browsing [Planet Explorer](https://www.planet.com/products/explorer/), filtering for 4 Band PlanetScope scene (`PSScene`) or Planetscope ortho tile (`PSOrthoTile`), and downloading an `ortho_analytic_4b` asset.

Before you download the full image, you can [preview a thumbnail](https://www.planet.com/docs/reference/data-api/previews/) of the image via Planet's Data API. (The thumbnails are 256x256 by default, and can be scaled up by passing in a `width` parameter.) 

In [ ]:
from IPython.display import Image
Image(url="https://api.planet.com/data/v1/item-types/PSScene/items/20170623_180038_0f34/thumb?width=512")

Next, you'll use [Planet's Data API](https://developers.planet.com/planetschool/searching-for-imagery-with-data-api/) to download the image.

In [ ]:
# Import helper modules
import json
import os
import requests
from planet import Auth, Session, DataClient
from pathlib import Path

In [ ]:
# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [ ]:
item_id_filter = {
"type":"StringInFilter",
"field_name":"id",
"config":["20170623_180038_0f34"]
}

item_types = ['PSScene']

In [ ]:
# Define our item_id:
item_id = '20170623_180038_0f34'

# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.quick_search(name='toar_planetscope',search_filter=item_id_filter, item_types=item_types)
    item_list = [i async for i in items]

In [ ]:
item = item_list[0]

In [ ]:
# By looking at the permissions, we can see the assets available for us to download
item['_permissions']

In [ ]:
# Get the assets link for the item
assets_url = item["_links"]["assets"]

# Print the assets link
print(assets_url)

In [ ]:
# Send a GET request to the assets url for the item (Get the list of available assets for the item)
res = session.get(assets_url)

# Assign a variable to the response
assets = res.json()

In [ ]:
# Assign a variable to the assets we want from the item's assets
orthoanalytic_asset = assets["ortho_analytic_4b"]
orthoanalytic_xml_asset = assets["ortho_analytic_4b_xml"]

# Print the assets data
p(orthoanalytic_asset)
p(orthoanalytic_xml_asset)

### Activating Assets

Before an asset is available for download, it must be activated.

In [ ]:
# Setup the activation url for a particular asset (in this case the basic_analytic_4b asset)
orthoanalytic_activation_url = orthoanalytic_asset["_links"]["activate"]
orthoanalytic_xml_activation_url = orthoanalytic_xml_asset["_links"]["activate"]

# Send a request to the activation url to activate the item
res_oa = session.get(orthoanalytic_activation_url)
res_oa_xml = session.get(orthoanalytic_xml_activation_url)


# Print the response from the activation request
p(res_oa.status_code)
p(res_oa_xml.status_code)

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

Below, we are polling the API until the item is done activation. This may take awhile.

In [ ]:
def activate_asset(assets_url, asset):
    asset_activated = False

    while asset_activated == False:
        # Send a request to the item's assets url
        res = session.get(assets_url)

        # Assign a variable to the item's assets url response
        assets = res.json()

        asset_status = asset["status"]
        print(asset_status)

        # If asset is already active, we are done
        if asset_status == 'active':
            asset_activated = True
            print("Asset is active and ready to download")

    # Print the ps3b_analytic asset data    
    return p(asset)

In [ ]:
# run this function on both of our assets
activate_asset(assets_url, orthoanalytic_asset)
activate_asset(assets_url, orthoanalytic_xml_asset)

### Downloading Assets

Click on the download links below to download each asset!

In [ ]:
# Assign a variable to the ortho_analytic asset's location endpoint
location_url = orthoanalytic_asset["location"]

# Print the location endpoint
print(location_url)

In [ ]:
# Assign a variable to the xml asset's location endpoint
location_url_xml = orthoanalytic_xml_asset["location"]

# Print the location endpoint
print(location_url_xml)

**Congratulations!** You now have two files in your `Downloads` directory: `20170623_180038_0f34_3B_AnalyticMS.tif` and `20170623_180038_0f34_3B_AnalyticMS_metadata.xml`. The first file is a GeoTIFF, the image you requested with spatial reference data embedded. The second file is a metadata file for that image that includes the data you'll need to convert from radiance to reflectance. 

## Step 2. Extract the Data from Each Spectral Band 

In this step, you'll use [Rasterio](https://github.com/mapbox/rasterio), a Python library for reading and writing geospatial raster datasets, to open the raster image you downloaded (the .tif file). Then you'll extract the data from the red and near-infrared bands and load the band data into arrays that you can manipulate using Python's [NumPy](http://www.numpy.org/) libary. *Note: in PlanetScope 4-band images, the band order is BGRN: (1) Blue, (2) Green, (3) Red, (4) Near-infrared.*

In [ ]:
downloads_path = str(Path.home() / "Downloads")

In [ ]:
import rasterio
import numpy as np

filename = os.path.join(downloads_path + "/20170623_180038_0f34_3B_AnalyticMS.tif")

# Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
with rasterio.open(filename) as src:
    band_blue_radiance = src.read(1)
    
with rasterio.open(filename) as src:
    band_green_radiance = src.read(2)

with rasterio.open(filename) as src:
    band_red_radiance = src.read(3)

with rasterio.open(filename) as src:
    band_nir_radiance = src.read(4)

## Step 3. Extract the Coefficients

Before you can convert to reflectance, you must extract the conversion coefficients from the metadata file you downloaded (the .xml file).

In [ ]:
from xml.dom import minidom
xml_path = os.path.join(downloads_path + "/20170623_180038_0f34_3B_AnalyticMS_metadata.xml")
xmldoc = minidom.parse(xml_path)
nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

# XML parser refers to bands by numbers 1-4
coeffs = {}
for node in nodes:
    bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
    if bn in ['1', '2', '3', '4']:
        i = int(bn)
        value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
        coeffs[i] = float(value)

print("Conversion coefficients: {}".format(coeffs))

Note that the coefficients are all of order `1e-5`, and that the coefficient for NIR is significantly higher than the coefficient for blue. This is a big deal if your use case involves performing band math because a pixel with a `NIR/blue` ratio of `1.0` in the radiance image will have a `NIR/blue` ratio of `3.35/1.929=1.73` in the reflectance image!

Most spectral indices are defined in terms of reflectance, not radiance.

## Step 4: Convert Radiance to Reflectance

Radiance is measured in SI units: W/m^2. Reflectance is a ratio from 0 to 1. The conversion is performed as a simple per-band scalar multiplication:

In [ ]:
# Multiply the Digital Number (DN) values in each band by the TOA reflectance coefficients
band_blue_reflectance = band_blue_radiance * coeffs[1]
band_green_reflectance = band_green_radiance * coeffs[2]
band_red_reflectance = band_red_radiance * coeffs[3]
band_nir_reflectance = band_nir_radiance * coeffs[4]

import numpy as np
print("Red band radiance is from {} to {}".format(np.amin(band_red_radiance), np.amax(band_red_radiance)))
print("Red band reflectance is from {} to {}".format(np.amin(band_red_reflectance), np.amax(band_red_reflectance)))

## Step 5. Save the Reflectance Image 

Next, you're going to save the calculated reflectance values to a new image file, making sure the new image file has the same geospatial metadata as the original GeoTIFF we downloaded.

In [ ]:
# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.uint16,
    count = 4)

print("Before Scaling, red band reflectance is from {} to {}".format(np.amin(band_red_reflectance),
                                                                      np.amax(band_red_reflectance)))

# Here we include a fixed scaling factor. This is common practice.
scale = 10000
blue_ref_scaled = scale * band_blue_reflectance
green_ref_scaled = scale * band_green_reflectance
red_ref_scaled = scale * band_red_reflectance
nir_ref_scaled = scale * band_nir_reflectance

print("After Scaling, red band reflectance is from {} to {}".format(np.amin(red_ref_scaled),
                                                                    np.amax(red_ref_scaled)))

# Write band calculations to a new raster file
with rasterio.open('data/reflectance.tif', 'w', **kwargs) as dst:
    dst.write_band(1, band_blue_reflectance.astype(rasterio.uint16))
    dst.write_band(2, band_green_reflectance.astype(rasterio.uint16))
    dst.write_band(3, band_red_reflectance.astype(rasterio.uint16))
    dst.write_band(4, band_nir_reflectance.astype(rasterio.uint16))

This warrants some explanation: Reflectance is generally defined as a floating point number between 0 and 1, but image file formats are much more commonly stored as unsigned integers. A common practice in the industry is to multiply the radiance value by 10,000, then save the result as a file with data type uint16.

## Step 6. Apply a Color Scheme to Visualize the Reflectance Values on the Image

In the last step, you'll use [Matplotlib](https://matplotlib.org/) to visualize the reflectance values you calculated for the PlanetScope scene.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

"""
The reflectance values will range from 0 to 1. You want to use a diverging color scheme to visualize the data,
and you want to center the colorbar at a defined midpoint. The class below allows you to normalize the colorbar.
"""

class MidpointNormalize(colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    Credit: Joe Kington, http://chris35wills.github.io/matplotlib_diverging_colorbar/
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))


# Set min/max values from reflectance range for image (excluding NAN)
min=np.nanmin(band_nir_reflectance)
max=np.nanmax(band_nir_reflectance)
mid=0.20

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)

# diverging color scheme chosen from https://matplotlib.org/users/colormaps.html
# note that appending '_r' to the color scheme name reverses it!
cmap = plt.cm.get_cmap('RdGy_r')

cax = ax.imshow(band_nir_reflectance, cmap=cmap, clim=(min, max), norm=MidpointNormalize(midpoint=mid,vmin=min, vmax=max))

ax.axis('off')
ax.set_title('NIR Reflectance', fontsize=18, fontweight='bold')

cbar = fig.colorbar(cax, orientation='horizontal', shrink=0.65)

fig.savefig("data/ref-fig.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

plt.show()

Looking at the NIR reflectance image, we see that the lakes and ponds appear to reflect almost no infrared light at all, but the healthy fields reflect almost 50% of the incoming infrared light!